# Phylogenetic Inference using PNUTS

This notebook gives an example on how to do phylogenetic inference using the PNUTS algorithm described in Wahle (2021) ([bioRxiv Paper](https://doi.org/10.1101/2021.03.16.435623))

First the `MCPhylo` package and the `Random` package are loaded.

In [ ]:
using MCPhylo;
using Random;
Random.seed!(1234);

The next step is to load the data.

In [ ]:
tree, data = make_tree_with_data("Example.nex");

The `tree` object contains a r random tree over the leaves specified in the nexus file. You can view a newick string representing the tree by calling the `newick` function on the object.

In [ ]:
newick(tree)

The input data needs to be stored in a dictionary to make it accessible to the sampler.

In [ ]:
data_dictionary = Dict{Symbol, Any}(
  :data => data
);

Define a model by specifing a prior distribution on the equilibrium frequencis, a Dirichlet prior in this case, and a prior on the phylogenetic tree. In this example the Compound Dirichlet distribution (Zhang, Rannala and Yang 2012. ([paper](https://doi.org/10.1093/sysbio/sys030))) is chosen.

The distribution associated with the data is the `PhyloDist`. It is a distribution whose likelihood function is calculated according to Felsensteins Pruning algorithm ([paper](https://doi.org/10.1007/BF01734359)).

The *Restriction Site Model* for character evolution and no rate variation accross sites is chosen.

In [ ]:
model =  Model(
    data = Stochastic(3, (tree, eq_freq) ->  PhyloDist(tree, eq_freq, [1.0], [1.0], Restriction), false, false),
    eq_freq = Stochastic(1, () -> Dirichlet(2,1)),
    tree = Stochastic(Node(), () -> CompoundDirichlet(1.0,1.0,0.100,1.0), true)
     )

Select the PNUTS sampler for the phylogenetic tree and the Slice sampler for the equilibrium frequency.

In [ ]:
scheme = [PNUTS(:tree, target=0.7, targetNNI=4),
          SliceSimplex(:eq_freq),
          ]
setsamplers!(model, scheme);

Set initial values.

In [ ]:
inits = [ Dict{Symbol, Union{Any, Real}}(
    :tree => tree,
    :eq_freq=> rand(Dirichlet(2,1)),
    :data => data_dictionary[:data]
    ),
    ];

Run the MCMC.

In [ ]:
sim = mcmc(model, data_dictionary, inits, 5000, burnin=2500,thin=5, chains=1, trees=true)